Pre

In [3]:
from Funcs.Utility import *
import numpy as np
import pandas as pd
from typing import Dict, Callable, Union, Tuple, List, Optional, Iterable
from datetime import timedelta as td
from scipy import stats
import ray
import warnings
import time
import ray
import dask
import scipy.spatial.distance as dist
import math

New modified Functions based on Panyu's original D1 preprocessing

In [4]:
import math
import numpy as np
import pandas as pd
from poi import PoiCluster
import googlemaps
import warnings
from pandas.errors import PerformanceWarning
from collections import defaultdict


warnings.simplefilter(action='ignore', category=PerformanceWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

# Define the center and radius of the circle around KAIST main campus
center_lat_kaist, center_lon_kaist = (36.3722, 127.3600)
_radius_kaist = 1000  # meters

# Define the center and radius of the circle around KAIST Munji campus
center_lat_munji, center_lon_munji = (36.391944, 127.398611)
_radius_munji = 400  # meters

# Define the center and radius of the circle around KAIST Seoul campus
center_lat_seoul, center_lon_seoul = (37.5933, 127.0464)
_radius_seoul = 300  # meters


def _haversine(_lat1, _lat2, _lng1, _lng2) -> float:
    if np.isnan(_lat1) or np.isnan(_lat2) or np.isnan(_lng1) or np.isnan(_lng2):
        return 0.0

    _lat1_r, _lat2_r, _lng1_r, _lng2_r = np.radians(_lat1), np.radians(_lat2), np.radians(_lng1), np.radians(_lng2)
    _lat = _lat2_r - _lat1_r
    _lng = _lng2_r - _lng1_r
    _R = 6371008.8
    _d = np.sin(_lat * 0.5) ** 2 + np.cos(_lat1_r) * np.cos(_lat2_r) * np.sin(_lng * 0.5) ** 2
    return 2 * _R * np.arcsin(np.sqrt(_d))

# Calculate the distances between the cluster centers and the center of the circle
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000 # meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(delta_lambda/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R*c
    return d

#Weiszfeld algorithm to calculate midpoint in a cluster
def midpoint(data: pd.DataFrame) -> pd.DataFrame:
    # Load the GPS locations of the cluster into a NumPy array
    _data = np.array(data[['latitude','longitude']])
    # Set the initial estimate to the mean of the GPS coordinates
    midpoint = np.mean(_data, axis=0)
    # Define the stopping criterion
    epsilon = 1e-6
    # Define the maximum number of iterations
    max_iterations = 100
    # Define the Weiszfeld algorithm
    for i in range(max_iterations):
        # Compute the distances between the midpoint and the points
        distances = np.sqrt(np.sum((_data - midpoint)**2, axis=1))
        
        # Check if any distance is 0
        if np.any(distances == 0):
            idx = np.where(distances == 0)[0][0]
            return pd.DataFrame({'mid_latitude': _data[idx, 0], 'mid_longitude': _data[idx, 1]}, index=data.index)
       
        
        # Check if the stopping criterion has been reached
        if np.max(distances) < epsilon:
            break
        # Compute the weighted mean of the GPS coordinates
        weights = 1.0 / distances
        midpoint = np.sum(_data * weights[:, np.newaxis], axis=0) / np.sum(weights)
    return pd.DataFrame({'mid_latitude': midpoint[0], 'mid_longitude': midpoint[1]}, index=data.index)  

In [ ]:
# Define the API key and client
API_KEY = "YOUR_API_KEY"
client = googlemaps.Client(API_KEY)

Eating = ['restaurant']
Social = ['bar','cafe','movie_theater','night_club']
Gym = ['gym']
known = Eating + Social + Gym

def label_cluster(data, radius):
    location = (data['mid_latitude'].mean(),data['mid_longitude'].mean())
    #return data['mid_latitude'].mean()
    # Perform the search
    results = client.places_nearby(location=location, radius=radius)
    # Filter the results by type (e.g. restaurant)
    known_places = [place for place in results['results'] if any(elem in place['types'] for elem in known)]
    if known_places:
        # Get the closest place to the location
        closest_place = min(known_places, key=lambda p: p.get('distance', {}).get('value', float('inf')))

        if any(elem in closest_place['types'] for elem in Social):
            data['label']='social'
        elif any(elem in closest_place['types'] for elem in Eating):
            data['label']='eating'
        else:
            data['label']='gym'
    else:
        data['label']='others'
    return data['label']

In [5]:
# ActivityEvent.csv (O)
def _proc_activity_event(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return {
        'UNK': data['confidenceUnknown'].astype('float32'),
        'FOT': data['confidenceOnFoot'].astype('float32'),
        'WLK': data['confidenceWalking'].astype('float32'),
        'VHC': data['confidenceInVehicle'].astype('float32'),
        'BCC': data['confidenceOnBicycle'].astype('float32'),
        'RUN': data['confidenceRunning'].astype('float32'),
        'TLT': data['confidenceTilting'].astype('float32')
    }

# ActivityTransition.csv (O)
def _proc_activity_transition(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    data = data.loc[
        lambda x: x['typeEnter'].isin(['WALKING', 'RUNNING', 'ON_BICYCLE', 'STILL', 'IN_VEHICLE']), :
    ]
    return data['typeEnter'].astype('object')

def _proc_wireless_state(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['type'].astype('object')

# FitnessActivity.csv (O)
def _proc_activity(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return {
        'VAL': data['value'].astype('object')
    }

# FitnessStepCount.csv (O)
def _proc_step_count(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return {
        'VAL': data['value'].astype('int')
    }

# FitnessDistance.csv (O)
def _proc_distance(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return {
        'VAL': data['value'].astype('float32')
    }

# FitnessCalorie.csv (O)
def _proc_calorie(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:

    return {
        'VAL': data['value'].astype('float32')
    }

def _proc_app_usage(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    # Filter the data to include only relevant types of app usage events
    data = data.loc[
        lambda x: x['type'].isin(['ACTIVITY_RESUMED', 'ACTIVITY_PAUSED']), :
    ].assign(
        category=lambda x: np.where(x['type'] == 'ACTIVITY_RESUMED', x['category'], None),
    )
    
    data = data.rename(columns={'category': 'subcategory'})
    data['category'] = [transform.get(item, item) for item in data['subcategory'].values]
    
    move = data
    Duration = []
    for pcode in data.index.get_level_values('pcode').unique():
        sub_move = move.loc[(pcode, ), :].sort_index(axis=0, level='timestamp').assign(pcode=pcode)
        sub_move = sub_move.reset_index()
        sub_move['move_state'] = sub_move['type'].shift().fillna('ACTIVITY_PAUSED')
        sub_move.loc[0, 'move_state'] = 'ACTIVITY_PAUSED'
        sub_move = sub_move[sub_move['move_state'] != sub_move['type']]
        sub_move.index = pd.to_datetime(sub_move.index)
        sub_move['duration'] = sub_move['timestamp'] - sub_move['timestamp'].shift()
        sub_move.loc[0, 'duration'] = pd.Timedelta(0)
        sub_move = sub_move[sub_move['duration'] > pd.Timedelta(0)]
        sub_move['duration_sec'] = sub_move['duration'].dt.total_seconds()
        sub_move['category'] = sub_move['category']
        Duration.append(sub_move)
    
    Duration = pd.concat(Duration, axis=0, ignore_index=True).set_index(
        ['pcode', 'timestamp']
    )
    Duration = Duration[Duration['type'] == 'ACTIVITY_RESUMED']
    
    cnt = Duration['category'].value_counts()
    _val, _sup = cnt.index, cnt.values
    
    DUR = {'DUR_{}'.format(_k): Duration[Duration['category'] == '{}'.format(_k)]['duration_sec'].astype('float32') for _k in _val}
    
    CAT = {'CAT': data['category'].astype('object')}
    
    Feature = {**DUR, **CAT}
    
    return Feature


def _proc_battery(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return {
        'LEV': data['level'].astype('float32'),
        'STA': data['status'].astype('object'),
        'TMP': (data['temperature'] / 10).astype('float32'),  # Assuming temperature is now in deci-degrees
        'PLG': data['pluggedType'].astype('object')
    }


def _proc_call(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    # Filter out calls with duration = 0
    data = data.loc[
        lambda x: x['duration'] > 0, :
    ]

    return {
        'DUR': data['duration'].astype('float32'),
        'CNT': data['contactType'].astype('object')
    }



def _proc_data_traffic(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return {
        'RCV': data['rxBytes'].astype('float32'),
        'SNT': data['txBytes'].astype('float32'),
        'MRCV': data['mobileRxBytes'].astype('float32'),
        'MSNT': data['mobileTxBytes'].astype('float32')
    }

def _proc_ringer_mode(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['type'].astype('object')

def _proc_screen(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    screen_events = data[data['type'].isin(['SCREEN_ON', 'SCREEN_OFF', 'USER_PRESENT'])]
    
    Duration = []
    for pcode in screen_events.index.get_level_values('pcode').unique():
        sub_screen = screen_events.loc[(pcode, ), :].sort_index(axis=0, level='timestamp').assign(pcode=pcode)
        sub_screen = sub_screen.reset_index()
        sub_screen['screen_state'] = sub_screen['type'].shift().fillna('SCREEN_OFF')
        sub_screen.loc[0, 'screen_state'] = 'SCREEN_OFF'
        sub_screen = sub_screen[sub_screen['screen_state'] != sub_screen['type']]
        sub_screen.index = pd.to_datetime(sub_screen.index)
        sub_screen['duration'] = sub_screen['timestamp'] - sub_screen['timestamp'].shift()
        sub_screen.loc[0, 'duration'] = pd.Timedelta(0)
        sub_screen = sub_screen[sub_screen['duration'] > pd.Timedelta(0)]
        sub_screen['duration_sec'] = sub_screen['duration'].dt.total_seconds()
        Duration.append(sub_screen)
    
    Duration = pd.concat(Duration, axis=0, ignore_index=True).set_index(['pcode', 'timestamp'])
    
    return {
        'EVENT': screen_events['type'].astype('object'),
        'DUR': Duration['duration_sec'].astype('float32')
    }

def _proc_on_off(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['type'].astype('object')

def _proc_power_save(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['type'].astype('object')

def _proc_charge(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['type'].astype('object')



def _proc_location(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    new_data = []
    DISTANCE_MAX_IN_METRE = 100  # Maximum distance in meters to be considered the same point of interest (POI)
    REGION_SIZE_IN_METRE = 250  # Maximum region size in meters for clustering POIs
    MAXIMUM_TIME_IN_MIN = 60  # Maximum time in minutes to be at a location
    MINIMUM_TIME_IN_MIN = 5  # Minimum time in minutes to be considered a visit at a location

    # Iterate through each unique pcode (user identifier)
    for pcode in data.index.get_level_values('pcode').unique():
        # Prepare subset of data for the current user
        sub = data.loc[(pcode,), :].sort_index(axis=0, level='timestamp').assign(
            _latitude=lambda x: x['latitude'].shift(1),  # Shift latitude to get the previous value
            _longitude=lambda x: x['longitude'].shift(1),  # Shift longitude to get the previous value
            dist=lambda x: x.apply(
                lambda y: _haversine(y['latitude'], y['_latitude'], y['longitude'], y['_longitude']),
                axis=1  # Calculate distance between current and previous coordinates
            ),
            pcode=pcode
        ).reset_index()

        # Convert timestamp to milliseconds
        sub['timestamp'] = sub['timestamp'].apply(lambda x: int(pd.Timestamp(x).timestamp() * 1000))
        
        # Filter out rows with high GPS accuracy values
        sub = sub[sub['accuracy'] < 100]

        # Convert latitude and longitude to radians for clustering
        latlon_rad = np.radians(sub.loc[:, ['latitude', 'longitude']].to_numpy())
        timestamps = sub.loc[:, 'timestamp'].values

        # Fit the POI clustering model
        cluster = PoiCluster(
            d_max=DISTANCE_MAX_IN_METRE, r_max=REGION_SIZE_IN_METRE, t_max=MAXIMUM_TIME_IN_MIN * 60 * 1000,
            t_min=MINIMUM_TIME_IN_MIN * 60 * 1000
        ).fit(X=latlon_rad, timestamps=timestamps)
        
        # Predict cluster labels for each point
        labels = cluster.predict(X=latlon_rad)
        sub = sub.assign(cluster=labels)

        # Replace empty cluster labels with 'NONE'
        sub['cluster'].replace('', 'NONE', inplace=True)

        # Calculate midpoint for each cluster
        sub[['mid_latitude', 'mid_longitude']] = sub.groupby('cluster', group_keys=False).apply(midpoint).apply(pd.Series)
        
        # Convert timestamp back to datetime and set time zone
        sub = sub.assign(
            _timestamp=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=True).dt.tz_convert(DEFAULT_TZ)
        )
        
        # Sort the DataFrame by timestamp
        sub = sub.sort_values(by='_timestamp')

        # # Extract temporal features
        # sub['day_of_week'] = sub['_timestamp'].dt.dayofweek  # Day of the week (0-6)
        # sub['hour_of_day'] = sub['_timestamp'].dt.hour  # Hour of the day (0-23)
        # sub['day_or_night'] = sub['_timestamp'].apply(lambda x: 1 if 9 <= x.hour < 18 else 0)  # Determine if it's day or night
        # sub['wkday_or_wkend'] = sub['_timestamp'].apply(lambda x: 1 if x.dayofweek <= 4 else 0)  # Determine if it's a weekday or weekend
        # sub['duration'] = sub['_timestamp'].diff().apply(lambda x: x.total_seconds() if isinstance(x, pd.Timedelta) else x)  # Calculate duration between consecutive timestamps
        # sub = sub.fillna(0)

        # # Identify home and work clusters
        # home = sub[sub['day_or_night'] == 0]['duration'].groupby(sub['cluster']).sum().idxmax()  # Cluster with most nighttime duration
        # work = sub[sub['day_or_night'] == 1][sub['wkday_or_wkend'] == 1][sub['cluster'] != home]['duration'].groupby(sub['cluster']).sum().idxmax()  # Cluster with most daytime weekday duration (excluding home)

        # # Label clusters as 'home', 'work', or 'none'
        # condition_home = sub['cluster'] == home
        # condition_work = sub['cluster'] == work
        # condition_none = sub['cluster'] == 'NONE'
        # sub.loc[condition_home, 'label'] = 'home'
        # sub.loc[condition_work, 'label'] = 'work'
        # sub.loc[condition_none, 'label'] = 'none'

        # # Label other clusters based on proximity using a specified radius
        # radius = 100
        # mask = sub['label'].isna()
        # sub['label'] = sub[mask].groupby('cluster', group_keys=False).apply(lambda x: label_cluster(x, radius)).apply(pd.Series)

        # # Re-label clusters as 'home', 'work', or 'none'
        # condition_home = sub['cluster'] == home
        # condition_work = sub['cluster'] == work
        # condition_none = sub['cluster'] == 'NONE'
        # sub.loc[condition_home, 'label'] = 'home'
        # sub.loc[condition_work, 'label'] = 'work'
        # sub.loc[condition_none, 'label'] = 'none'

        # # Calculate distances to specific known locations (e.g., KAIST, Munji, Seoul)
        # centers = sub[sub['label'] == 'others'].groupby('cluster').mean()
        # distances_kaist = centers.apply(lambda row: haversine(center_lat_kaist, center_lon_kaist, row['latitude'], row['longitude']), axis=1)
        # distances_munji = centers.apply(lambda row: haversine(center_lat_munji, center_lon_munji, row['latitude'], row['longitude']), axis=1)
        # distances_seoul = centers.apply(lambda row: haversine(center_lat_seoul, center_lon_seoul, row['latitude'], row['longitude']), axis=1)

        # # Determine which clusters fall within specified radii of known locations
        # in_circle_kaist = distances_kaist <= _radius_kaist
        # cluster_centers_in_circle_kaist = centers[in_circle_kaist]

        # in_circle_munji = distances_munji <= _radius_munji
        # cluster_centers_in_circle_munji = centers[in_circle_munji]

        # in_circle_seoul = distances_seoul <= _radius_seoul
        # cluster_centers_in_circle_seoul = centers[in_circle_seoul]

        # # Combine clusters within known locations
        # cluster_centers_in_circle = pd.concat([cluster_centers_in_circle_kaist, cluster_centers_in_circle_munji, cluster_centers_in_circle_seoul])

        # # Label clusters that are within known locations as 'work'
        # condition_work_other = sub['cluster'].isin(cluster_centers_in_circle.index)
        # sub.loc[condition_work_other, 'label'] = 'work'

        # Append processed data for the current user
        new_data.append(sub)

    # Concatenate data from all users
    new_data = pd.concat(new_data, axis=0, ignore_index=True)

    # Convert timestamp back to datetime with the default time zone
    new_data['timestamp'] = pd.to_datetime(new_data['timestamp'], unit='ms', utc=True).dt.tz_convert(DEFAULT_TZ)
    new_data = new_data.set_index(['pcode', 'timestamp'])

    # Return the labeled location data
    return {
        # 'LABEL': new_data['label'].astype('object'),
        'DST': new_data['dist'].astype('float32'),
        'CLS': new_data['cluster'].astype('object'),

        
    }




def _proc_wifi(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    new_data = []

    for pcode in data.index.get_level_values('pcode').unique():
        sub = data.loc[(pcode,), :].sort_index(axis=0, level='timestamp').assign(
            bssid=lambda x: x['address'].str.cat(x['frequency'].astype(str), sep='-')
        )
        t = sub.index.unique().array
        for cur_t, prev_t in zip(t, t[1:]):
            if cur_t is pd.NaT or prev_t is pd.NaT:
                continue

            prev = sub.loc[[prev_t], :]
            cur = sub.loc[[cur_t], :]

            # Ensure bssid is unique by averaging RSSI values
            prev = prev.groupby('bssid', as_index=False).agg({'rssi': 'mean'})
            cur = cur.groupby('bssid', as_index=False).agg({'rssi': 'mean'})

            # Create the union of BSSIDs
            union_bssids = np.union1d(prev['bssid'], cur['bssid'])

            # Create aligned RSSI values for prev and cur based on the union of bssids
            prev_rssi = prev.set_index('bssid').reindex(union_bssids)['rssi'].fillna(-100)
            cur_rssi = cur.set_index('bssid').reindex(union_bssids)['rssi'].fillna(-100)

            # Ensure the lengths match for cosine similarity calculation
            if len(prev_rssi) != len(cur_rssi):
                continue  # Skip this comparison if lengths don't match (though reindex should guarantee matching)

            new_data.append(dict(
                pcode=pcode,
                timestamp=cur_t,
                cosine=1 - dist.cosine(prev_rssi, cur_rssi) if len(union_bssids) > 0 else 0,
                euclidean=1 / (1 + dist.euclidean(prev_rssi, cur_rssi)) if len(union_bssids) > 0 else 0,
                manhattan=1 / (1 + dist.cityblock(prev_rssi, cur_rssi)) if len(union_bssids) > 0 else 0,
                jaccard=len(np.intersect1d(prev['bssid'], cur['bssid'])) / len(union_bssids) if len(union_bssids) > 0 else 0
            ))

    new_data = pd.DataFrame(new_data).set_index(['pcode', 'timestamp'])
    
    return {
        'COS': new_data['cosine'].astype('float32'),
        'EUC': new_data['euclidean'].astype('float32'),
        'MAN': new_data['manhattan'].astype('float32'),
        'JAC': new_data['jaccard'].astype('float32')
    }

def _proc_installed_app(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    new_data = []
    
    for pcode in data.index.get_level_values('pcode').unique():
        sub = data.loc[(pcode, ), :].sort_index(axis=0, level='timestamp')
        t = sub.index.unique().array
        for cur_t, prev_t in zip(t, t.shift(1)):
            if cur_t is pd.NaT or prev_t is pd.NaT:
                continue

            prev = sub.loc[[prev_t], :]
            cur = sub.loc[[cur_t], :]
            intersect = np.intersect1d(prev['packageName'], cur['packageName'])
            union = np.union1d(prev['packageName'], cur['packageName'])
            new_data.append(dict(
                pcode=pcode,
                timestamp=cur_t,
                jaccard=len(intersect) / len(union) if len(union) > 0 else 0
            ))
            
    new_data = pd.DataFrame(new_data).set_index(['pcode', 'timestamp'])
    
    return {
       'JAC': new_data['jaccard'].astype('float32')
    }


def _proc_message_event(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    new_data = defaultdict(list)
    
    for pcode in data.index.get_level_values('pcode').unique():
        sub = data.loc[(pcode, ), :].sort_index(
            axis=0, level='timestamp'
        )

        sent = sub.loc[
            lambda x: x['messageBox'] == 'SENT', :
        ].assign(
            event=1,
            pcode=pcode
        ).reset_index()

        recv = sub.loc[
            lambda x: x['messageBox'] == 'INBOX', :
        ].assign(
            event=1,
            pcode=pcode
        ).reset_index()

        msg = sub.assign(
            event=1,
            pcode=pcode
        ).reset_index()

        new_data['SNT'].append(sent)
        new_data['RCV'].append(recv)
        new_data['ALL'].append(msg)

    return {
        k: pd.concat(
            v, axis=0, ignore_index=True
        ).set_index(
            ['pcode', 'timestamp']
        )['event'].astype('float32') 
        for k, v in new_data.items()
    }




def _proc_bluetooth_scan(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    # Simply return the bondState, deviceType, and classType as they are
    return { 
        'BondState' : data['bondState'].astype('object'),
        'DeviceType': data['deviceType'].astype('object'), 
        'classType': data['classType'].astype('object'), 
    }

def _proc_notification_event(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return {
        'VIS': data['visibility'].astype('object'),
        'CAT': data['category'].astype('object')
    }


def _proc_dozemode_event(data: pd.DataFrame) -> pd.Series:
    # Ensure the data is sorted by timestamp
    data = data.sort_values(by='timestamp')
    return data['type'].astype('object')

def _proc_fitbit_heartrate(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['value'].astype('float32')

def _proc_fitbit_stepcount(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['value'].astype('float32')

def _proc_fitbit_calorie(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['value'].astype('float32')

def _proc_fitbit_distance(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    return data['value'].astype('float32')


def _proc_key(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    data = data.rename(columns={'category': 'subcategory'})
    data['category'] = [transform.get(item, item) for item in data['subcategory'].values]
    return {'CAT': data['category'].astype('object'), 
            'DIST': data['distance'].astype('float32'),
            'TIME': data['timeTaken'].astype('float32')}

In [6]:
import os
import pandas as pd
import gc
from functools import reduce
import warnings
from pandas.errors import PerformanceWarning
import ray

warnings.simplefilter(action='ignore', category=PerformanceWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

In [7]:
FUNC_PROC = {
    # 'ActivityEvent': _proc_activity_event, #newly added
    # 'ActivityTransition': _proc_activity_transition, #newly added
    # 'WirelessStateEvent': _proc_wireless_state, #newly added
    # 'Fitness-Calorie': _proc_calorie,
    # 'Fitness-Activity': _proc_activity, #newly added
    # 'Fitness-Distance': _proc_distance, #newly added
    # 'Fitness-StepCount': _proc_step_count, #newly added
    # 'AppUsageEvent': _proc_app_usage,
    # 'BatteryEvent': _proc_battery,
    # 'CallEvent': _proc_call,
    # 'DataTraffic': _proc_data_traffic,
    # 'InstalledApp': _proc_installed_app,
    'Location': _proc_location,
    # 'MessageEvent': _proc_message_event,
    # 'WifiScan': _proc_wifi,
    # 'ScreenEvent': _proc_screen,
    # 'RingerModeEvent': _proc_ringer_mode,
    # 'ChargeEvent': _proc_charge,
    # 'PowerSaveEvent': _proc_power_save,
    # 'OnOffEvent': _proc_on_off,
    # 'BluetoothScan': _proc_bluetooth_scan,
    # 'DozeModeEvent': _proc_dozemode_event,
    # 'Fitbit-HeartRate': _proc_fitbit_heartrate,  
    # 'Fitbit-StepCount': _proc_fitbit_stepcount, #newly added
    # 'Fitbit-Calorie':_proc_fitbit_calorie, #newly added
    # 'Fitbit-Distance':_proc_fitbit_distance, #newly added
    # 'KeyEvent': _proc_key, #newly added  
    # 'NotificationEvent': _proc_notification_event
}

In [8]:
import os
import pandas as pd
import gc
from typing import Optional, Union, Dict
import numpy as np
import ray
from collections import defaultdict
from poi import PoiCluster 
from Funcs.Utility import _load_data


def _process(data_type: str):
    log(f'Begin to processing data: {data_type}')
    
    abbrev = DATA_TYPES[data_type]
    data_raw = _load_data(data_type)
    
    # Debugging: Check if data_raw is empty
    if data_raw.empty:
        print(f"No data loaded for {data_type}")
        log(f"No data loaded for {data_type}")
        return {}
    
    data_proc = FUNC_PROC[data_type](data_raw)
    result = dict()
    
    if type(data_proc) is dict:
        for k, v in data_proc.items():
            result[f'{abbrev}_{k}'] = v
    else:
        result[abbrev] = data_proc
        
    log(f'Complete processing data: {data_type}')
    return result


In [9]:
with on_ray():
    jobs = []
    
    func = ray.remote(_process).remote
    
    for data_type in DATA_TYPES:
        job = func(data_type)
        jobs.append(job)

    jobs = ray.get(jobs)
    
    jobs = reduce(lambda a, b: {**a, **b}, jobs)
    dump(jobs, os.path.join(PATH_INTERMEDIATE, 'proc_loc_v2.pkl'))

    del jobs
    gc.collect()

2024-11-12 15:33:50,538	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(_process pid=1879819) [24-11-12 15:33:51] Begin to processing data: Location
(_process pid=1879819) Paths for Location: [('P126', '/var/nfs_share/D#4/newdata/P126/Location.csv'), ('P041', '/var/nfs_share/D#4/newdata/P041/Location.csv'), ('P008', '/var/nfs_share/D#4/newdata/P008/Location.csv'), ('P026', '/var/nfs_share/D#4/newdata/P026/Location.csv'), ('P065', '/var/nfs_share/D#4/newdata/P065/Location.csv'), ('P124', '/var/nfs_share/D#4/newdata/P124/Location.csv'), ('P116', '/var/nfs_share/D#4/newdata/P116/Location.csv'), ('P123', '/var/nfs_share/D#4/newdata/P123/Location.csv'), ('P091', '/var/nfs_share/D#4/newdata/P091/Location.csv'), ('P040', '/var/nfs_share/D#4/newdata/P040/Location.csv'), ('P038', '/var/nfs_share/D#4/newdata/P038/Location.csv'), ('P078', '/var/nfs_share/D#4/newdata/P078/Location.csv'), ('P061', '/var/nfs_share/D#4/newdata/P061/Location.csv'), ('P043', '/var/nfs_share/D#4/newdata/P043/Location.csv'), ('P075', '/var/nfs_share/D#4/newdata/P075/Location.csv'), ('P007',

(_process pid=1879819) /tmp/ipykernel_1874141/1420200091.py:170: PerformanceWarning: indexing past lexsort depth may impact performance.
(_process pid=1879819) /tmp/ipykernel_1874141/1420200091.py:170: PerformanceWarning: indexing past lexsort depth may impact performance.
(_process pid=1879819) /tmp/ipykernel_1874141/1420200091.py:170: PerformanceWarning: indexing past lexsort depth may impact performance.
(_process pid=1879819) /tmp/ipykernel_1874141/1420200091.py:170: PerformanceWarning: indexing past lexsort depth may impact performance.
(_process pid=1879819) /tmp/ipykernel_1874141/1420200091.py:170: PerformanceWarning: indexing past lexsort depth may impact performance.
(_process pid=1879819) /tmp/ipykernel_1874141/1420200091.py:170: PerformanceWarning: indexing past lexsort depth may impact performance.
(_process pid=1879819) /tmp/ipykernel_1874141/1420200091.py:170: PerformanceWarning: indexing past lexsort depth may impact performance.
(_process pid=1879819) /tmp/ipykernel_187

(_process pid=1879819) [24-11-12 15:34:08] Complete processing data: Location


In [12]:
data_loc_v2 = pd.read_pickle(os.path.join(PATH_INTERMEDIATE, 'proc_loc_v2.pkl'))

In [15]:
data_loc_v2

{'LOC_DST': pcode  timestamp                       
 P126   2021-12-06 15:30:30.622000+09:00      0.000000
        2021-12-06 15:32:16.025000+09:00     75.783302
        2021-12-06 15:34:15.170000+09:00     78.214386
        2021-12-06 15:39:25.419000+09:00     16.788128
        2021-12-06 15:41:02.878000+09:00     13.604548
                                               ...    
 P083   2021-12-28 23:33:06.771000+09:00      5.711549
        2021-12-28 23:39:04.200000+09:00      7.898235
        2021-12-28 23:45:01.310000+09:00      5.523997
        2021-12-28 23:45:36.581000+09:00      5.960837
        2021-12-28 23:51:40.129000+09:00    101.850624
 Name: dist, Length: 612941, dtype: float32,
 'LOC_CLS': pcode  timestamp                       
 P126   2021-12-06 15:30:30.622000+09:00      NONE
        2021-12-06 15:32:16.025000+09:00      NONE
        2021-12-06 15:34:15.170000+09:00      NONE
        2021-12-06 15:39:25.419000+09:00      NONE
        2021-12-06 15:41:02.878000+09:00  

In [14]:
data_loc = pd.read_pickle(os.path.join(PATH_INTERMEDIATE, 'proc_loc.pkl'))

In [18]:
data_except_loc = pd.read_pickle(os.path.join(PATH_INTERMEDIATE, 'proc_except_loc.pkl'))

In [21]:
dump({**data_loc, **data_loc_v2, **data_except_loc}, os.path.join(PATH_INTERMEDIATE, 'proc_updated.pkl'))